# MNIST Trial 2: Creating shifted copies of the images and using them to train models

In [1]:
# loading the model
from sklearn.datasets import fetch_openml
MNIST = fetch_openml("mnist_784", version=1, as_frame=False)

In [3]:
MNIST.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [4]:
X, y = MNIST['data'], MNIST['target']

In [5]:
import numpy as np

In [6]:
y = y.astype(np.uint8)

In [7]:
X_train, X_test, y_train, y_test = X[:60_000], X[60_000:], y[:60_000], y[60_000:]

## Creating a fucntion to shift the image

In [10]:
from scipy.ndimage.interpolation import shift
from matplotlib import pyplot as plt

In [9]:
some_digit = X_train[0]

In [11]:
# a function to draw the digit
def  draw_digit(digit):
    digit_image = np.reshape(digit, (28, 28))
    plt.imshow(digit_image, cmap='binary')
    plt.show()

In [29]:
def make_shifts(digit, units_to_shift):
    # function for making the shifts
    digit_image = np.reshape(digit, (28, 28))
    shifted_digit = shift(digit_image, units_to_shift)
    return shifted_digit.reshape(784)

In [33]:
# making the shifts
X_train_shifted_right = np.apply_along_axis(make_shifts, 1, X_train, units_to_shift=[0, 1])
X_train_shifted_left = np.apply_along_axis(make_shifts, 1, X_train, units_to_shift=[0, -1])
X_train_shifted_down = np.apply_along_axis(make_shifts, 1, X_train, units_to_shift=[1, 0])
X_train_shifted_up = np.apply_along_axis(make_shifts, 1, X_train, units_to_shift=[-1, 0])

In [37]:
# combining all the datasets
X_train_updated = np.concatenate((X_train_shifted_right, 
                                  X_train_shifted_left, 
                                  X_train_shifted_down, 
                                  X_train_shifted_up), axis=0)
y_train_updated = np.concatenate((y_train,
                                  y_train,
                                  y_train,
                                  y_train), axis=0)

## Training Classifiers with the new dataset

In [41]:
import pickle

def train_and_save_classifier(classifier_type, name, x, y, **kwargs):
    # function for training and saving model
    classifier =  classifier_type(**kwargs)
    classifier.fit(x, y)
    file_path = 'saved_models/saved_models_trial2/' + name + '.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump(classifier, file)
        
def load_classifier(name):
    # function for loading saved models
    file_path = 'saved_models/saved_models_trial2/' + name + '.pkl'
    with open(file_path, 'rb') as file:
        model = pickle.load(file)
    return model

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_predict, cross_val_score

def evaluate_classifier(classifier, x, y, **kwargs):
    # function for evaluating classifiers with some selected matrics
    print("Accuracies: ", cross_val_score(classifier, x, y, cv=3, scoring="accuracy"))
    predicted_values = cross_val_predict(classifier, x, y, cv=3)
    conf_matrix = confusion_matrix(y, predicted_values)
    print("\nConfusion Metrix: ")
    print(conf_matrix)
    
    print("\nPrecision Score: ", precision_score(y, predicted_values, **kwargs))
    print("Recall Score: ", recall_score(y, predicted_values, **kwargs))
    print("f1 Score: ", f1_score(y, predicted_values, **kwargs))
    
    print("\nROC AUC Score: ", roc_auc_score(y, predicted_values, **kwargs))

### SVM Classifier

In [43]:
from sklearn.svm import SVC
train_and_save_classifier(SVC, 'sv_classifier', X_train_updated, y_train_updated, random_state=42)